<a href="https://colab.research.google.com/github/MinyaevAndrey/ultralytics-supervision-pet-/blob/main/notebooks/ultralytics-solutions/01_how_to_count_the_objects_using_ultralytics_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Counting using Ultralytics YOLO11

This notebook serves as a starting point for [counting objects](https://docs.ultralytics.com/guides/object-counting/) in videos or live streams using the YOLO11 model.

### What is Object Counting?

- Object counting with YOLO11 involves accurate identification and counting of specific objects in videos and camera streams. YOLO11 excels in real-time applications, providing efficient and precise object counting for various scenarios like crowd analysis and surveillance, thanks to its state-of-the-art algorithms and deep learning capabilities.

### Advantages of Object Counting?

- **Resource Optimization**: Object counting facilitates efficient resource management by providing accurate counts, and optimizing resource allocation in applications like inventory management.
- **Enhanced Security**: Object counting enhances security and surveillance by accurately tracking and counting entities, aiding in proactive threat detection.
- **Informed Decision-Making**: Object counting offers valuable insights for decision-making, optimizing processes in retail, traffic management, and various other domains.

In [ ]:
a=5
a*5

5

In [ ]:
!pip install ultralytics -q

import ultralytics
import cv2
from ultralytics import solutions

ultralytics.checks()

###№

Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/112.6 GB disk)


### Read the Video File

- You can either read the video file directly or stream the content from an RTSP (Real-Time Streaming Protocol) source, allowing for flexible video input depending on your needs.
- We will also set up the video writer to handle the output video writing.

In [ ]:
# !pip install -q ultralytics supervision
# import supervision as sv

# !wget https://media.roboflow.com/supervision/video-examples/vehicles.mp4
# print(sv.VideoInfo.from_video_path(video_path='/content/vehicles.mp4'))
# !ffmpeg -y -i /content/vehicles.mp4 -vf "scale=iw/4:ih/4" -c:a copy /content/vehicles-2.mp4 > /dev/null 2>&1
# print(sv.VideoInfo.from_video_path(video_path='/content/vehicles-2.mp4'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 6.2 MB/s eta 0:00:00
--2025-05-25 07:50:09--  https://media.roboflow.com/supervision/video-examples/vehicles.mp4
Resolving media.roboflow.com (media.roboflow.com)... 34.110.133.209
Connecting to media.roboflow.com (media.roboflow.com)|34.110.133.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35345757 (34M) [video/mp4]
Saving to: ‘vehicles.mp4’

vehicles.mp4        100%[===================>]  33.71M   103MB/s    in 0.3s    

2025-05-25 07:50:10 (103 MB/s) - ‘vehicles.mp4’ saved [35345757/35345757]

VideoInfo(width=3840, height=2160, fps=25, total_frames=538)
VideoInfo(width=960, height=540, fps=25, total_frames=538)


In [ ]:
cap = cv2.VideoCapture("/content/vehicles-2.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH,
                                       cv2.CAP_PROP_FRAME_HEIGHT,
                                       cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("counting.avi",
                               cv2.VideoWriter_fourcc(*"mp4v"),
                               fps, (w, h))

# region_points = [[87, 302], [871, 290]]
region_points = [[201, 254], [733, 244], [957, 349], [58, 367]] # в зоне въезд-выезд считается по крайним линиям

counter = solutions.ObjectCounter(
    # show=True,  # Display the output
    region=region_points,  # Pass region points
    model="/content/yolo11n.engine",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    verbose = False,  # Display the output
    # classes=[0, 2],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    show_in=True,  # Display in counts
    show_out=True,  # Display out counts
    line_width=1,  # Adjust the line width for bounding boxes and text display
    device=0,
    tracker='bytetrack.yaml'
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    results = counter(im0)  # count the objects
    video_writer.write(results.plot_im)   # write the video frames

cap.release()   # Release the capture
video_writer.release()
cv2.destroyAllWindows()

Ultralytics Solutions: ✅ {'source': None, 'model': '/content/yolo11n.engine', 'classes': None, 'show_conf': True, 'show_labels': True, 'region': [[201, 254], [733, 244], [957, 349], [58, 367]], 'colormap': 21, 'show_in': True, 'show_out': True, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'figsize': (12.8, 7.2), 'blur_ratio': 0.5, 'vision_point': (20, 20), 'crop_dir': 'cropped-detections', 'json_file': None, 'line_width': 1, 'records': 5, 'fps': 30.0, 'max_hist': 5, 'meter_per_pixel': 0.05, 'max_speed': 120, 'show': False, 'iou': 0.7, 'conf': 0.25, 'device': 0, 'max_det': 300, 'half': False, 'tracker': 'bytetrack.yaml', 'verbose': False, 'data': 'images'}
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/yolo11n.engine for TensorRT inference...
WARNING ⚠️ no tracks found!
WARNING ⚠️ no tracks found!
WARNING ⚠

In [ ]:
!ffmpeg -y -i /content/counting.avi -c:v libx264 -c:a aac -strict experimental -b:a 192k /content/counting.mp4 > /dev/null 2>&1

In [ ]:
import moviepy.editor
scale = 1
video_info = sv.VideoInfo.from_video_path(video_path='/content/counting.mp4')
moviepy.editor.ipython_display('/content/counting.mp4',
                               width=video_info.width * scale,
                               height = video_info.height*scale)

Crafted with 💙 by [Ultralytics](https://ultralytics.com/)  

🌟 Explore and star the [Ultralytics Notebooks](https://github.com/ultralytics/notebooks/) to supercharge your AI journey! 🚀